<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA to Parquet

_by Joe Hamman (CarbonPlan), June 30, 2020_

This notebook converts FIA csv files to Parquet format and stages them in a
Google Cloud Storage bucket.

**Inputs:**

- `ENTIRE` directory

**Outputs:**

- One Parquet dataset per CSV: `gs://carbonplan-data/raw/fia/<name>.parquet`

**Notes:**

- No reprojection or processing of the data is done in this notebook.


In [ ]:
import io
import os.path
import pathlib

import gcsfs
import pandas as pd

from carbonplan_data.utils import setup

# run `gcloud auth login` on the command line, or try switching token to `browser`
fs = gcsfs.GCSFileSystem(
    project="carbonplan",
    token="/Users/jhamman/.config/gcloud/legacy_credentials/joe@carbonplan.org/adc.json",
)

In [ ]:
workdir, upload = setup("joe")

In [ ]:
csvs = (workdir / "fia/ENTIRE").glob("*csv")

In [ ]:
import numpy as np


def force_float32(fname):

    memmap = fname.stat().st_size > 1e8

    df = pd.read_csv(fname, engine="c", low_memory=False, memory_map=memmap)
    for c in df:
        if "f8" in df[c].dtype.str:
            df[c] = df[c].astype(np.float32)

    return df

In [ ]:
def exists(blob):
    try:
        f = fs.open(blob, "rb")
        f.close()
        return True
    except:
        return False

In [ ]:
failed = []
for fname in csvs:
    blob = f"carbonplan-data/raw/fia/{fname.stem}.parquet"
    print(fname.stem)

    if "TREE.csv" in str(fname):
        continue

    if exists(blob):
        continue

    df = force_float32(fname)

    print(blob)

    try:
        df.to_parquet(
            blob,
            compression="gzip",
            open_with=fs.open,
            row_group_offsets=1000,
            engine="fastparquet",
        )
        # consider using dask dataframe here to write to chunked dataframes here.
        print("  --> ", blob)
    except Exception as e:
        failed.append((fname, e))

In [ ]:
failed

In [ ]:
# TREE.csv is a special case

import dask.dataframe as dd

row_group_offsets = 1000
dtype = {
    "AGENTCD": "float64",
    "CULL": "float64",
    "P2A_GRM_FLG": "object",
    "TREECLCD": "float64",
    "TREEHISTCD": "float64",
    "MODIFIED_IN_INSTANCE": "float64",
    "GST_PNWRS": "object",
    "SPGRPCD": "float64",
    "DIAHTCD": "float64",
    "SUBCYCLE": "float64",
    "CAVITY_USE_PNWRS": "object",
}

blob = "TREE.parquet"

df = dd.read_csv(
    "/Users/jhamman/workdir/carbonplan_data_downloads/fia/ENTIRE/TREE.csv",
    dtype=dtype,
)

df.to_parquet("gs://carbonplan-data/raw/fia/TREE.parquet")